In [8]:
%cd /content/drive/MyDrive/myproject

/content/drive/MyDrive/myproject


In [9]:


!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 21.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)

In [10]:
!pip install langchain
!pip install flask_cors
!pip install langchain_groq

In [11]:
!pip install dotenv

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [12]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import random
from collections import Counter
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import os
from flask import Flask, request, jsonify, render_template
# from flask_ngrok import run_with_ngrok  # Ensure flask_ngrok is installed
from flask_cors import CORS
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from tenacity import retry, stop_after_attempt, wait_exponential
import time
load_dotenv()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


False

In [13]:
# Load the GROQ and OpenAI API keys

groq_api_key = "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"

# Initialize the ChatGroq model
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

llm
print(groq_api_key)

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


In [14]:
prompt = ChatPromptTemplate.from_template(
    """
    Use the provided context to answer the question accurately and concisely.
    Only include information that is directly relevant to the question.
    <context>
    {context}
    <context>
    Question: {input}
    """
)

In [15]:
from sentence_transformers import SentenceTransformer
# Ensure necessary packages are installed

In [16]:

# from tenacity import retry, stop_after_attempt, wait_exponential

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.7 MB/s eta 0:00:00


In [17]:
# Define a function to download and return SentenceTransformer embeddings
def download_huggingface_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [18]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [19]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)  # Wrap extracted_data in a list
    return text_chunks

In [20]:
# @retry(wait=wait_exponential(multiplier=1, min=4, max=60), stop=stop_after_attempt(6))
def vector_embedding():
    embeddings = download_huggingface_embeddings()
    # loader = load_pdf("./data")  # Load PDF files from the directory
    # docs = loader.load()  # Document loading
    extracted_data = load_pdf("./data")
    # extracted_data_hair = load_pdf("./data/HairTips.pdf")
    # docs = extracted_data_skin + extracted_data_hair
    final_documents=text_split(extracted_data)
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  # Chunk creation
    # final_documents = text_splitter.split_documents(docs[:20])  # Splitting documents into chunks
    vectors = FAISS.from_documents(final_documents, embeddings)  # Creating vector embeddings
    return vectors, final_documents

In [21]:
def print_documents(docs):
    for doc in docs:
        print("Document content:")
        for line in doc.page_content.split('\n'):
            print(line)
        print("-" * 40)

In [22]:
vectors, final_documents = vector_embedding()
print("The vectors are:", vectors)

# print("Raw Documents:")
# print_documents(docs)

print("\nProcessed Document Chunks:")
print_documents(final_documents)
# Print details about the FAISS vector store
print("\nFAISS Vector Store Details:")
print(f"Number of documents: {len(final_documents)}")
print(f"Embedding dimension: {vectors.index.d}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The vectors are: <langchain_community.vectorstores.faiss.FAISS object at 0x7f91642f0820>

Processed Document Chunks:
Document content:
General Hair Care Tips  
Regular Washing:  
 
Frequency: Wash your hair regularly to keep the scalp clean. 
For oily hair, washing every other day is recommended, while 
dry hair may need washing just 2 -3 times a week.  
Product: Use a gentle shampoo suitable for your hair type. 
Avoid sulfates if your hair is dry or color -treated.  
Conditioning:  
 
Type: Use a conditioner that matches your hair type to maintain 
moisture balance.
----------------------------------------
Document content:
moisture balance.  
Application: Apply conditioner mainly to the ends of your hair, 
avoiding the scalp to preve nt greasiness.  
Hair Drying:  
 
Towel Drying: Pat your hair dry with a soft towel. Avoid rubbing 
as it can cause breakage and frizz.  
Air Drying: Whenever possible, let your hair air dry to minimize 
heat damage.  
Heat Protection:  
 
Products: Use 

In [23]:
conversation_history = []

def ask_question(question):
    conversation_history.append(f"User: {question}")
    context = "\n".join(conversation_history)

    document_chain = create_stuff_documents_chain(llm, prompt)
    retriever = vectors.as_retriever()
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    response = retrieval_chain.invoke({'input': question, 'context': context})
    answer = response['answer']

    conversation_history.append(f"Assistant: {answer}")
    return answer

In [24]:
question = input("Enter a question: ")
response = ask_question(question)
print("Answer:", response)

Enter a question: what is head lice
Answer: Head lice are small, wingless, blood-sucking insects that live in the hair on your head and feed off the blood from your scalp.


In [25]:
train="/content/dive/MyDrive/skin-disease-datasaet/train_set"
test="/content/dive/MyDrive/skin-disease-datasaet/test_set"
train
test

'/content/dive/MyDrive/skin-disease-datasaet/test_set'

In [26]:

import tensorflow as tf
from tensorflow.keras import backend as K
@tf.keras.saving.register_keras_serializable()
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
@tf.keras.saving.register_keras_serializable()
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
@tf.keras.saving.register_keras_serializable()
def f1_score(y_true, y_pred):
    precision_value = precision(y_true, y_pred)
    recall_value = recall(y_true, y_pred)
    f1_score = 2 * (precision_value * recall_value) / (precision_value + recall_value + K.epsilon())
    return f1_score
tf.keras.utils.get_custom_objects().update({"recall": recall, "precision": precision, "f1_score": f1_score})


In [27]:
!pip install Flask-Session
!pip install flask_session
!pip install ngrok
!pip install flask_cors
!pip install pyngrok
!pip install flask-ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.6 MB/s eta 0:00:00


In [28]:
from flask import Flask, request, jsonify, render_template, session
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from flask_session import Session  # Add this import


In [29]:
# # %cd /content/drive/MyDrive/myproject
# from flask import Flask, request, jsonify, render_template, session
# from flask_ngrok import run_with_ngrok
# from flask_cors import CORS
# from pyngrok import ngrok
# # from flask_session import Session  # Import Session from flask_session

# app = Flask(__name__)
# # run_with_ngrok(app)  # Start ngrok when app is run
# CORS(app)
# # app.config["SESSION_TYPE"] = "filesystem"
# # Session(app)


# @app.route("/demos")
# def demos():
#     return "helooo"

# @app.route("/")
# def indexsss():
#     return render_template('chat.html')

# @app.route("/get", methods=["GET", "POST"])
# def chatsss():
#     msg = request.json.get("msg", "")  # Use JSON payload
#     query = msg
#     print("Input is:", query)

#     # Retrieve the predicted disease from session
#     # predicted_disease = session.get("predicted_disease", None)
#     # if predicted_disease:
#     #     query = f"{query} The predicted disease is {predicted_disease}."

#     answer = ask_question(query)
#     response = {"response": answer}

#     print("response is", response)
#     return jsonify(response)

# # Load the saved model
# model_path = "/content/drive/MyDrive/skin-disease-datasaet/skin_disease_vgg16_model.keras"
# model = load_model(model_path, custom_objects={"recall": recall, "precision": precision, "f1_score": f1_score})

# # Define the labels
# labels = ['BA-cellulitis', 'BA-impetigo', 'FU-athlete-foot', 'FU-nail-fungus', 'FU-ringworm', 'PA-cutaneous-larva-migrans', 'VI-chickenpox', 'VI-shingles']

# # Create a label encoder and fit it with the labels
# le = LabelEncoder()
# le.fit(labels)

# @app.route('/predict', methods=['POST'])
# def predict():
#     if 'file' not in request.files:
#         return jsonify({'error': 'No file part'})

#     file = request.files['file']

#     if file.filename == '':
#         return jsonify({'error': 'No selected file'})

#     if file:
#         filename = "uploaded_image.png"
#         file.save(filename)

#         def preprocess_image(image_path):
#             """Preprocess the input image for model prediction."""
#             img = cv2.imread(image_path)
#             img = cv2.resize(img, (224, 224))
#             img = preprocess_input(np.array([img]))  # Add an extra dimension for batching
#             return img

#         def predict_disease(image_path):
#             """Predict the disease name from the input image."""
#             img = preprocess_image(image_path)
#             predictions = model.predict(img)
#             predicted_class_index = np.argmax(predictions)
#             predicted_class = le.inverse_transform([predicted_class_index])[0]
#             return predicted_class

#         predicted_disease = predict_disease(filename)
#         print("Predicted disease is", predicted_disease)

#         # Save the predicted disease in the session
#         session["predicted_disease"] = predicted_disease

#         return jsonify({'disease_name': predicted_disease[3:]})

# if __name__ == '__main__':
#     ngrok.set_auth_token("2grl4UzpnpzlYdL7YlR72G5WHWy_3Ze8QshboF2wu2tGzKb2i")
#     public_url = ngrok.connect(5000)
#     print(" * ngrok URL:", public_url)
#     app.run(use_reloader=False)


In [30]:
from flask import Flask, request, jsonify, render_template, session
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from pyngrok import ngrok
import joblib
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input

app = Flask(__name__)
CORS(app)

# Load the skin disease model and label encoder
skin_model_path = "/content/drive/MyDrive/skin-disease-datasaet/skin_disease_vgg16_model.keras"
skin_model = load_model(skin_model_path, custom_objects={"recall": recall, "precision": precision, "f1_score": f1_score})

skin_labels = ['BA-cellulitis', 'BA-impetigo', 'FU-athlete-foot', 'FU-nail-fungus', 'FU-ringworm', 'PA-cutaneous-larva-migrans', 'VI-chickenpox', 'VI-shingles']
skin_le = LabelEncoder()
skin_le.fit(skin_labels)

# Load the hair disease model and label encoder
hair_model_path = '/content/drive/MyDrive/HairDiseases/hair_disease_vgg16_model.keras'
hair_model = load_model(hair_model_path)

hair_le = joblib.load('/content/drive/MyDrive/HairDiseases/label_encoder.pkl')

# Preprocess image function
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(np.array([img], dtype=np.float32))  # Add batch dimension
    return img

# Predict disease function
def predict_disease(image_path, model, label_encoder):
    img = preprocess_image(image_path)
    predictions = model.predict(img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
    return predicted_class

@app.route("/demos")
def demos():
    return "helooo"


@app.route("/")
def index():
    return render_template('chat.html')

@app.route("/get", methods=["GET", "POST"])
def chat():
    msg = request.json.get("msg", "")  # Use JSON payload
    query = msg
    print("Input is:", query)

    answer = ask_question(query)  # Ensure this function is defined
    response = {"response": answer}

    print("response is", response)
    return jsonify(response)

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        filename = "uploaded_image.png"
        file.save(filename)

        # Get the prediction type (skin or hair)
        prediction_type = request.form.get("type", "").lower()
        if prediction_type == "skin":
            predicted_disease = predict_disease(filename, skin_model, skin_le)
        elif prediction_type == "hair":
            predicted_disease = predict_disease(filename, hair_model, hair_le)
        else:
            return jsonify({'error': 'Invalid prediction type'})

        print("Predicted disease is", predicted_disease)
        if predicted_disease == "undefined":
            return jsonify({"error": "Please upload a valid image"})
        else:
            return jsonify({'disease_name': predicted_disease})

if __name__ == '__main__':
    ngrok.set_auth_token("2grl4UzpnpzlYdL7YlR72G5WHWy_3Ze8QshboF2wu2tGzKb2i")
    public_url = ngrok.connect(5000)
    print(" * ngrok URL:", public_url)
    app.run(use_reloader=False)

 * ngrok URL: NgrokTunnel: "https://10f5-34-66-210-200.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 16:23:58] "OPTIONS /get HTTP/1.1" 200 -


Input is: what is head lice


INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 16:23:59] "POST /get HTTP/1.1" 200 -


response is {'response': 'Head lice are small, wingless, blood-sucking insects that live in the hair on your head and feed off the blood from your scalp.'}


...

Predicted disease is PA-cutaneous-larva-migrans


INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 16:25:59] "OPTIONS /get HTTP/1.1" 200 -


Input is: tell me more about above mentioned disease. Predicted Disease: PA-cutaneous-larva-migrans


INFO:werkzeug:127.0.0.1 - - [19/Jun/2024 16:26:00] "POST /get HTTP/1.1" 200 -


response is {'response': 'Here\'s what I gathered from the context:\n\n**What is it?**\nCutaneous Larva Migrans (CLM) is a skin condition caused by several species of parasites. It\'s also known as "creeping eruption" or "larva migrans".\n\n**Symptoms**\nLesions on the feet, backside, and other areas likely to be exposed to contaminated soil or sand. Can occur anywhere on the body, but most often on areas like feet, buttocks, thighs, and hands.\n\n**Treatment**\nCLM is self-limiting, meaning the larvae under the skin typically die off within 5-6 weeks without treatment. However, in some cases, it may take longer for the infection to go away. Medications can help speed up the healing process.\n\n**Prevention**\nTo prevent infection, wear shoes, clothing that covers the thighs and buttocks, and avoid walking barefoot or sitting on the ground without a barrier. CLM is most common in warm areas like the southeastern United States, the Caribbean, Central and South America, Africa, and South